In [1]:
from observer import CalibratedObserver, TrackedObject
import dma.MechaCombat as mc
QuantumSystem = mc.QuantumSystem

In [3]:
class HarmonyMachine(CalibratedObserver):
    def commitChanges(self, trackedObj):
        super().commitChanges(trackedObj)

In [ ]:
class GameState:
    states = ["Add", "Movement", "Declare", "Resolve"]
    state = "Add"
    round = 0
    declaredActions = {}

    @classmethod
    def reset(cls):
        cls.round = 0
        cls.state = "Add"
        cls.declaredActions = {}

    @classmethod
    def nextState(cls, currentState=None):
        if currentState is not None:
            assert currentState == cls.state, "GameState mismatch. Cannot progress in this manner"
        currentState = cls.state
        newState = currentState
        if newState == "Add":
            newState = "Movement"
            cls.movements = []
        elif newState == "Movement":
            newState = "Declare"
            cls.declaredActions = {}
        elif newState == "Declare":
            newState = "Resolve"
        elif newState == "Resolve":
            newState = "Movement"
            cls.declaredActions = {}
            cls.round += 1
        cls.state = newState